Sets up the necessary dependencies and language models for using spaCy in Spanish and Catalan

In [ ]:
!pip install -U spacy
!python -m spacy download es_core_news_md
!python -m spacy download ca_core_news_md

In [ ]:
!pip install python-crfsuite

In [88]:
import spacy
import json
import pycrfsuite as crfs

Parse the JSON train and test files

In [89]:
def parse_json(file_path):

  # Step 2: Open the file in read mode
  try:
    with open(file_path, "r") as json_file:

      # Step 3: Load the JSON data using json.load()
      parsed_file = json.load(json_file)
  except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
  else:
    print("JSON data parsed successfully!")
    # Step 4: Access and process the data
    # (See examples below based on data structure)
  return parsed_file

In [90]:
training_set = parse_json("./train_data.json")
test_set = parse_json("./test_data.json")

JSON data parsed successfully!
JSON data parsed successfully!


In [91]:
print(training_set[3]['data']['text'])

 nº historia clinica: ** *** *** nºepisodi: ******** sexe: home data de naixement: 14.03.1953 edat: 66 anys procedencia domicil/res.soc servei oncologia medica data d'ingres 25.03.2019 data d'alta 02.04.2019 12:30:00 ates per ************, *****; *******************, ***************** informe d'alta d'hospitalitzacio motiu d'ingres paciente varon de 66 años que acude a urgencias por vomitos y mareos. antecedents no alergias medicamentosas. historia oncologica: afecto de glioblastoma medular . reseccion parcial. se decide qt -rdt esquema stupp tmz y rdt hasta dt 60 gy rt /qt stupp temozolomida. --&gt; fin 23.2.18 1º ciclo de tmz 6/4/18, con rmn a los 3 ciclos sd. damos un total de 7 ciclos con buena tolerancia. pd a los 7 ciclos en forma de diseminacion medular no signisficativa clinicamente. se decid e re-irradiacion con tmz concomitante como radiosensibilizador . evento tv: tvp eeii bilateral proximal abril/18. eco-doppler 24.07.18: venas femorales permeables compresibles con fascidad

In [92]:
print(len(training_set))

254


Get words information from every training set texts  
We skip the texts containing inconsistency words(eg: ex-fumador)

In [93]:
skip_list = [16,45,106,116,119,120,163,173,177,179,198,215,216,226,229,230,236,243,248]

In [ ]:
text_words_info = []
nlp = spacy.load('es_core_news_md')  # Load the Spanish language model
for i in range(len(training_set)):
  if i in skip_list:
    continue
  text = training_set[i]['data']['text']

  # Parse the text
  doc = nlp(text)

  # Extract sentences
  sentences = [sent.text for sent in doc.sents]
  print("text_number:", i+1)

  # Extract words with start and end positions, lemma, and POS tagging for each word
  words_info = [{"sent_idx":idx, "sent":sent,"word":token.text, "start":token.idx, "end":token.idx + len(token), "lemma":token.lemma_, "pos":token.pos_,"tag":' ',"prefix":token.prefix_,"suffix":token.suffix_}
                for idx,sent in enumerate(doc.sents) for token in sent if not token.is_space and token.text not in ["*", "(",",", "?",",","!",":",";",".","&","\",","/","-"]]
  text_words_info.append(words_info)

Preprocess test data

In [95]:
skip_list_test = [3,13,21]

In [ ]:
text_words_info_test = []
nlp = spacy.load('es_core_news_md')  # Load the Spanish language model
for i in range(len(test_set)):
  if i in skip_list_test:
    continue
  text = test_set[i]['data']['text']

  # Parse the text
  doc = nlp(text)

  # Extract sentences
  sentences = [sent.text for sent in doc.sents]
  print("text_number:", i+1)

  # Extract words with start and end positions, lemma, and POS tagging for each word
  words_info = [{"sent_idx":idx, "sent":sent,"word":token.text, "start":token.idx, "end":token.idx + len(token), "lemma":token.lemma_, "pos":token.pos_,"tag":' ',"prefix":token.prefix_,"suffix":token.suffix_}
                for idx,sent in enumerate(doc.sents) for token in sent if not token.is_space and token.text not in ["*", "(",",", "?",",","!",":",";",".","&","\",","/","-"]]
  text_words_info_test.append(words_info)

In [97]:
print(len(text_words_info))

235


In [98]:
print(len(text_words_info_test))

61


 Extract and sort annotations labeled as "NEG", "NSCO", "UNC", and "USCO" from training set.

In [99]:
text_neg = []
text_nsco = []
text_unc = []
text_usco = []
for idx,document in enumerate(training_set):
  if idx in skip_list:
    continue
  negations = [result_element['value'] for result_element in document["predictions"][0]["result"] if "NEG" in result_element["value"]["labels"]]
  negations = sorted(negations, key=lambda x: x['start'])
  nsco = [result_element['value'] for result_element in document["predictions"][0]["result"] if "NSCO" in result_element["value"]["labels"]]
  nsco = sorted(nsco, key=lambda x: x['start'])
  unc = [result_element['value'] for result_element in document["predictions"][0]["result"] if "UNC" in result_element["value"]["labels"]]
  unc = sorted(unc, key=lambda x: x['start'])
  usco = [result_element['value'] for result_element in document["predictions"][0]["result"] if "USCO" in result_element["value"]["labels"]]
  usco = sorted(usco, key=lambda x: x['start'])
  text_neg.append(negations)
  text_nsco.append(nsco)
  text_unc.append(unc)
  text_usco.append(usco)



In [100]:
print(len(text_neg))
print(len(text_nsco))
print(len(text_unc))
print(len(text_usco))

235
235
235
235


 Extract and sort annotations labeled as "NEG", "NSCO", "UNC", and "USCO" from test set.

In [101]:
text_neg_test = []
text_nsco_test = []
text_unc_test = []
text_usco_test = []
for idx,document in enumerate(test_set):
  if idx in skip_list_test:
    continue
  negations = [result_element['value'] for result_element in document["predictions"][0]["result"] if "NEG" in result_element["value"]["labels"]]
  negations = sorted(negations, key=lambda x: x['start'])
  nsco = [result_element['value'] for result_element in document["predictions"][0]["result"] if "NSCO" in result_element["value"]["labels"]]
  nsco = sorted(nsco, key=lambda x: x['start'])
  unc = [result_element['value'] for result_element in document["predictions"][0]["result"] if "UNC" in result_element["value"]["labels"]]
  unc = sorted(unc, key=lambda x: x['start'])
  usco = [result_element['value'] for result_element in document["predictions"][0]["result"] if "USCO" in result_element["value"]["labels"]]
  usco = sorted(usco, key=lambda x: x['start'])
  text_neg_test.append(negations)
  text_nsco_test.append(nsco)
  text_unc_test.append(unc)
  text_usco_test.append(usco)



In [102]:
print(len(text_neg_test))
print(len(text_nsco_test))
print(len(text_unc_test))
print(len(text_usco_test))

61
61
61
61


This code tags words in a document with negation labels ("B-NEG", "I-NEG", "E-NEG").

For "NEG" - Train set

In [103]:
for negs, words_info in zip(text_neg, text_words_info):
  neg_counter=0
  words_counter =0
  while neg_counter < len(negs):

    #move to the next word if didn't find the beginning of a negation
    lst = ['exfumador','exfumadora','ex-fumador','ex-fumadora']
    if words_info[words_counter]['word'] in lst:
      words_info[words_counter]['tag'] = 'B-NEG'
      neg_counter+=1
      words_counter+=1
      continue
    if abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) > 1:
      words_counter+=1

    #the beginning of a negation is the same as its end
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <=1:
      words_info[words_counter]['tag'] = 'B-NEG'
      neg_counter+=1
      words_counter+=1

    #the negation has more than one word
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:
      words_info[words_counter]['tag'] = 'B-NEG'
      words_counter+=1

      #tag the inside of the negation
      while abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:
        words_info[words_counter]['tag'] = 'I-NEG' # comment this for ignoring inside tagging
        words_counter+=1

      #tag the end of it
      if abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <= 1:
        words_info[words_counter]['tag'] = 'E-NEG'
        neg_counter+=1





For "NEG" - Test set

In [104]:
for negs, words_info in zip(text_neg_test, text_words_info_test):
  neg_counter=0
  words_counter =0
  while neg_counter < len(negs):

    #move to the next word if didn't find the beginning of a negation
    lst = ['exfumador','exfumadora','ex-fumador','ex-fumadora']
    if words_info[words_counter]['word'] in lst:
      words_info[words_counter]['tag'] = 'B-NEG'
      neg_counter+=1
      words_counter+=1
      continue
    if abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) > 1:
      words_counter+=1

    #the beginning of a negation is the same as its end
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <=1:
      words_info[words_counter]['tag'] = 'B-NEG'
      neg_counter+=1
      words_counter+=1

    #the negation has more than one word
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:
      words_info[words_counter]['tag'] = 'B-NEG'
      words_counter+=1

      #tag the inside of the negation
      while abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:
        words_info[words_counter]['tag'] = 'I-NEG' # comment this for ignoring inside tagging
        words_counter+=1

      #tag the end of it
      if abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <= 1:
        words_info[words_counter]['tag'] = 'E-NEG'
        neg_counter+=1





For "NSCO" - Train set

In [150]:
for negs, words_info in zip(text_nsco, text_words_info):
  neg_counter=0
  words_counter =0
  while neg_counter < len(negs):

    #move to the next word if didn't find the beginning of a negation
    lst = ['exfumador','exfumadora','ex-fumador','ex-fumadora']
    if words_info[words_counter]['word'] in lst:
      words_info[words_counter]['tag'] = 'B-NSCO'
      neg_counter+=1
      words_counter+=1
      continue
    if abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) > 1:
      words_counter+=1

    #the beginning of a negation is the same as its end
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <=1:
      words_info[words_counter]['tag'] = 'B-NSCO'
      neg_counter+=1
      words_counter+=1

    #the negation has more than one word
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:
      words_info[words_counter]['tag'] = 'B-NSCO'
      words_counter+=1

      #tag the inside of the negation
      while abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:

        words_info[words_counter]['tag'] = 'I-NSCO' # comment this for ignoring inside tagging
        words_counter+=1

      #tag the end of it
      if abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <= 1:
        words_info[words_counter]['tag'] = 'E-NSCO'
        neg_counter+=1


For "NSCO" - Test Set

In [106]:
for negs, words_info in zip(text_nsco_test, text_words_info_test):
  neg_counter=0
  words_counter =0
  while neg_counter < len(negs):

    #move to the next word if didn't find the beginning of a negation
    lst = ['exfumador','exfumadora','ex-fumador','ex-fumadora']
    if words_info[words_counter]['word'] in lst:
      words_info[words_counter]['tag'] = 'B-NSCO'
      neg_counter+=1
      words_counter+=1
      continue
    if abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) > 1:
      words_counter+=1

    #the beginning of a negation is the same as its end
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <=1:
      words_info[words_counter]['tag'] = 'B-NSCO'
      neg_counter+=1
      words_counter+=1

    #the negation has more than one word
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:
      words_info[words_counter]['tag'] = 'B-NSCO'
      words_counter+=1

      #tag the inside of the negation
      while abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:

        words_info[words_counter]['tag'] = 'I-NSCO' # comment this for ignoring inside tagging
        words_counter+=1

      #tag the end of it
      if abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <= 1:
        words_info[words_counter]['tag'] = 'E-NSCO'
        neg_counter+=1


For "UNC" - Train Set

In [107]:
for negs, words_info in zip(text_unc, text_words_info):
  neg_counter=0
  words_counter =0
  while neg_counter < len(negs):


    if abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) > 1:
      words_counter+=1

    #the beginning of a negation is the same as its end
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <=1:
      words_info[words_counter]['tag'] = 'B-UNC'
      neg_counter+=1
      words_counter+=1

    #the negation has more than one word
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:
      words_info[words_counter]['tag'] = 'B-UNC'
      words_counter+=1

      #tag the inside of the negation
      while abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:
        words_info[words_counter]['tag'] = 'I-UNC' # comment this for ignoring inside tagging
        words_counter+=1

      #tag the end of it
      if abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <= 1:
        words_info[words_counter]['tag'] = 'E-UNC'
        neg_counter+=1

For "UNC" - Train Set

In [108]:
for negs, words_info in zip(text_unc_test, text_words_info_test):
  neg_counter=0
  words_counter =0
  while neg_counter < len(negs):


    if abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) > 1:
      words_counter+=1

    #the beginning of a negation is the same as its end
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <=1:
      words_info[words_counter]['tag'] = 'B-UNC'
      neg_counter+=1
      words_counter+=1

    #the negation has more than one word
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:
      words_info[words_counter]['tag'] = 'B-UNC'
      words_counter+=1

      #tag the inside of the negation
      while abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:
        words_info[words_counter]['tag'] = 'I-UNC' # comment this for ignoring inside tagging
        words_counter+=1

      #tag the end of it
      if abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <= 1:
        words_info[words_counter]['tag'] = 'E-UNC'
        neg_counter+=1

For "USCO" - Train Set

In [109]:
for negs, words_info in zip(text_usco, text_words_info):
  neg_counter=0
  words_counter =0
  while neg_counter < len(negs):

    if abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) > 1:
      words_counter+=1

    #the beginning of a negation is the same as its end
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <=1:
      words_info[words_counter]['tag'] = 'B-USCO'
      neg_counter+=1
      words_counter+=1

    #the negation has more than one word
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:
      words_info[words_counter]['tag'] = 'B-USCO'
      words_counter+=1

      #tag the inside of the negation
      while abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:
        words_info[words_counter]['tag'] = 'I-USCO' # comment this for ignoring inside tagging
        words_counter+=1

      #tag the end of it
      if abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <= 1:
        words_info[words_counter]['tag'] = 'E-USCO'
        neg_counter+=1

For "USCO" - Test Set

In [110]:
for negs, words_info in zip(text_usco_test, text_words_info_test):
  neg_counter=0
  words_counter =0
  while neg_counter < len(negs):

    if abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) > 1:
      words_counter+=1

    #the beginning of a negation is the same as its end
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <=1:
      words_info[words_counter]['tag'] = 'B-USCO'
      neg_counter+=1
      words_counter+=1

    #the negation has more than one word
    elif abs(negs[neg_counter]['start'] - words_info[words_counter]['start']) <=1 and abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:
      words_info[words_counter]['tag'] = 'B-USCO'
      words_counter+=1

      #tag the inside of the negation
      while abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) > 1:
        words_info[words_counter]['tag'] = 'I-USCO' # comment this for ignoring inside tagging
        words_counter+=1

      #tag the end of it
      if abs(negs[neg_counter]['end'] - words_info[words_counter]['end']) <= 1:
        words_info[words_counter]['tag'] = 'E-USCO'
        neg_counter+=1

Tag the word outside of any named entity as 'O' - Train set

In [111]:
for text in text_words_info:
  for word_info in text:
    if word_info['tag']==' ':
      word_info['tag'] = 'O'

Tag the word outside of any named entity as 'O' - Test set

In [112]:
for text in text_words_info_test:
  for word_info in text:
    if word_info['tag']==' ':
      word_info['tag'] = 'O'

In [ ]:
for word_info in text_words_info[0]:
  print(word_info)

 Iterates through each document and groups words into sentences, appending each sentence to `text_sentences`. - Train set

In [114]:
text_sentences = []
for document in text_words_info:
  sentences = []
  sentence = []
  current_idx = 1
  for word in document:
    if word['sent_idx'] == current_idx:
      sentence.append(word)
    else:
      sentences.append(sentence)
      current_idx += 1
      sentence = [word]
  text_sentences.append(sentences)

 Iterates through each document and groups words into sentences, appending each sentence to `text_sentences`. - Test set

In [115]:
text_sentences_test = []
for document in text_words_info_test:
  sentences = []
  sentence = []
  current_idx = 1
  for word in document:
    if word['sent_idx'] == current_idx:
      sentence.append(word)
    else:
      sentences.append(sentence)
      current_idx += 1
      sentence = [word]
  text_sentences_test.append(sentences)

In [116]:
print(text_sentences[0][0])

[{'sent_idx': 1, 'sent': nº historia clinica: ** *** *** nºepisodi: ******** sexe: home data de naixement: 16.05.1936 edat: 82 anys procedencia cex mateix hosp servei urologia data d'ingres 24.07.2018 data d'alta 25.07.2018 08:54:04 ates per ***************, *****; ****************, ****** informe d'alta d'hospitalitzacio motiu d'ingres paciente que ingresa de forma programada para realizacion de uretrotomia interna ., 'word': 'nº', 'start': 1, 'end': 3, 'lemma': 'nº', 'pos': 'NOUN', 'tag': 'O', 'prefix': 'n', 'suffix': 'nº'}, {'sent_idx': 1, 'sent': nº historia clinica: ** *** *** nºepisodi: ******** sexe: home data de naixement: 16.05.1936 edat: 82 anys procedencia cex mateix hosp servei urologia data d'ingres 24.07.2018 data d'alta 25.07.2018 08:54:04 ates per ***************, *****; ****************, ****** informe d'alta d'hospitalitzacio motiu d'ingres paciente que ingresa de forma programada para realizacion de uretrotomia interna ., 'word': 'historia', 'start': 4, 'end': 12, 'l

Print some words in a sentence for first text

In [117]:
for text in text_sentences_test[:1]:
  for sentence in text:
    words = []
    for word in sentence:
      words.append(word['word'])
    print(words)
    print("--------sentence ends --------")
  print(' ')
  print("-------text ends --------")
  print(' ')
print("?")

['nº', 'historia', 'clinica', 'nºepisodi', 'sexe', 'dona', 'data', 'de', 'naixement', '12.05.1977', 'edat', '42', 'anys', 'procedencia', 'aguts', 'servei', 'obstetricia', 'data', "d'ingres", '27.09.2019', 'data', "d'alta", '01.10.2019', '13:00:00', 'ates', 'per', 'informe', "d'alta", "d'hospitalitzacio", 'motiu', "d'ingres", 'induccion', 'al', 'parto', 'por', 'pequeño', 'para', 'la', 'edad', 'gestacional', 'peg', ')', 'antecedents', 'no', 'alergias', 'medicamentosas', 'conocidas', 'antcededentes', 'medico-quirurgicos', 'protesis', 'mamaria', 'adenoidectomia', 'niega', 'habitos', 'toxicos', 'medicacio', 'habitual', 'anafranil25', 'mg/', 'diario']
--------sentence ends --------
['yodocefol']
--------sentence ends --------
['hierro', 'oral']
--------sentence ends --------
['ranitidina', '150', 'mg']
--------sentence ends --------
['proces', 'actual', 'edad', '42', 'años', 'tpal', '1001', 'eutocico', ')', 'fur', '21.12.2019', 'sg', '40', '+', '0', 'gsrh', 'a+', 'gestacion', 'controlada', '

Flattens `text_sentences` into a single list of sentences - Train set

In [118]:
#Compute a list of sentences with every sentence of the text_sentences
sentences = []
for text in text_sentences:
  for sentence in text:
    words_info = []
    for word_info in sentence:
      del word_info['sent_idx']
      del word_info['sent']
      words_info.append(word_info)
    sentences.append(words_info)

Flattens `text_sentences` into a single list of sentences - Test set

In [119]:
#Compute a list of sentences with every sentence of the text_sentences
sentences_test = []
for text in text_sentences_test:
  for sentence in text:
    words_info = []
    for word_info in sentence:
      del word_info['sent_idx']
      del word_info['sent']
      words_info.append(word_info)
    sentences_test.append(words_info)

Train Set

In [120]:
print(sentences[4])
print(len(sentences))

[{'word': 'diverticulosis', 'start': 558, 'end': 572, 'lemma': 'diverticulosi', 'pos': 'PRON', 'tag': 'O', 'prefix': 'd', 'suffix': 'sis'}, {'word': 'extensa', 'start': 573, 'end': 580, 'lemma': 'extenso', 'pos': 'ADJ', 'tag': 'O', 'prefix': 'e', 'suffix': 'nsa'}, {'word': 'insuficiencia', 'start': 581, 'end': 594, 'lemma': 'insuficiencia', 'pos': 'NOUN', 'tag': 'O', 'prefix': 'i', 'suffix': 'cia'}, {'word': 'renal', 'start': 595, 'end': 600, 'lemma': 'renal', 'pos': 'ADJ', 'tag': 'O', 'prefix': 'r', 'suffix': 'nal'}, {'word': 'cronica', 'start': 601, 'end': 608, 'lemma': 'cronico', 'pos': 'ADJ', 'tag': 'O', 'prefix': 'c', 'suffix': 'ica'}, {'word': 'colelitiasis', 'start': 609, 'end': 621, 'lemma': 'colelitiasis', 'pos': 'NOUN', 'tag': 'O', 'prefix': 'c', 'suffix': 'sis'}, {'word': 'antecedentes', 'start': 622, 'end': 634, 'lemma': 'antecedente', 'pos': 'NOUN', 'tag': 'O', 'prefix': 'a', 'suffix': 'tes'}, {'word': 'quirurgicos', 'start': 635, 'end': 646, 'lemma': 'quirurgico', 'pos': 

Test set

In [121]:
print(sentences_test[4])
print(len(sentences_test))

[{'word': 'proces', 'start': 607, 'end': 613, 'lemma': 'proces', 'pos': 'NOUN', 'tag': 'O', 'prefix': 'p', 'suffix': 'ces'}, {'word': 'actual', 'start': 614, 'end': 620, 'lemma': 'actual', 'pos': 'ADJ', 'tag': 'O', 'prefix': 'a', 'suffix': 'ual'}, {'word': 'edad', 'start': 621, 'end': 625, 'lemma': 'edad', 'pos': 'NOUN', 'tag': 'O', 'prefix': 'e', 'suffix': 'dad'}, {'word': '42', 'start': 627, 'end': 629, 'lemma': '42', 'pos': 'NUM', 'tag': 'O', 'prefix': '4', 'suffix': '42'}, {'word': 'años', 'start': 630, 'end': 634, 'lemma': 'año', 'pos': 'NOUN', 'tag': 'O', 'prefix': 'a', 'suffix': 'ños'}, {'word': 'tpal', 'start': 635, 'end': 639, 'lemma': 'tpal', 'pos': 'ADJ', 'tag': 'O', 'prefix': 't', 'suffix': 'pal'}, {'word': '1001', 'start': 641, 'end': 645, 'lemma': '1001', 'pos': 'NUM', 'tag': 'O', 'prefix': '1', 'suffix': '001'}, {'word': 'eutocico', 'start': 647, 'end': 655, 'lemma': 'eutocico', 'pos': 'ADJ', 'tag': 'O', 'prefix': 'e', 'suffix': 'ico'}, {'word': ')', 'start': 655, 'end':

Processes a list of sentences to count the occurrences of different tags for each word. - Train set

In [122]:
total_words = 0
bneg = 0
bnsco = 0
ineg = 0
insco = 0
eneg = 0
ensco = 0
bunc = 0
busco = 0
iunc = 0
iusco = 0
eunc = 0
eusco = 0
o = 0

for sentence in sentences:
  for word in sentence:
    total_words += 1
    if word['tag'] == 'O':
      o += 1
    elif word['tag'] == 'B-NEG':
      bneg += 1
    elif word['tag'] == 'B-NSCO':
      bnsco += 1
    elif word['tag'] == 'B-UNC':
      bunc += 1
    elif word['tag'] == 'B-USCO':
      busco += 1
    elif word['tag'] == 'I-NEG':
      ineg += 1
    elif word['tag'] == 'I-NSCO':
      insco += 1
    elif word['tag'] == 'I-UNC':
      iunc += 1
    elif word['tag'] == 'I-USCO':
      iusco += 1
    elif word['tag'] == 'E-NEG':
      eneg += 1
    elif word['tag'] == 'E-NSCO':
      ensco += 1
    elif word['tag'] == 'E-UNC':
      eunc += 1
    elif word['tag'] == 'E-USCO':
      eusco += 1

print(f"Total words: {total_words}")

print("\nB-Tags:")
print(f"  B-NEG:  {bneg}")
print(f"  B-NSCO: {bnsco}")
print(f"  B-UNC:  {bunc}")
print(f"  B-USCO: {busco}")

print("\nI-Tags:")
print(f"  I-NEG:  {ineg}")
print(f"  I-NSCO: {insco}")
print(f"  I-UNC:  {iunc}")
print(f"  I-USCO: {iusco}")

print("\nE-Tags:")
print(f"  E-NEG:  {eneg}")
print(f"  E-NSCO: {ensco}")
print(f"  E-UNC:  {eunc}")
print(f"  E-USCO: {eusco}")

print(f"\nO-Tags: {o}")

Total words: 163485

B-Tags:
  B-NEG:  3877
  B-NSCO: 3713
  B-UNC:  418
  B-USCO: 411

I-Tags:
  I-NEG:  0
  I-NSCO: 5253
  I-UNC:  4
  I-USCO: 879

E-Tags:
  E-NEG:  76
  E-NSCO: 2511
  E-UNC:  198
  E-USCO: 319

O-Tags: 145826


Processes a list of sentences to count the occurrences of different tags for each word. - Test set

In [123]:
total_words = 0
bneg = 0
bnsco = 0
ineg = 0
insco = 0
eneg = 0
ensco = 0
bunc = 0
busco = 0
iunc = 0
iusco = 0
eunc = 0
eusco = 0
o = 0

for sentence in sentences_test:
  for word in sentence:
    total_words += 1
    if word['tag'] == 'O':
      o += 1
    elif word['tag'] == 'B-NEG':
      bneg += 1
    elif word['tag'] == 'B-NSCO':
      bnsco += 1
    elif word['tag'] == 'B-UNC':
      bunc += 1
    elif word['tag'] == 'B-USCO':
      busco += 1
    elif word['tag'] == 'I-NEG':
      ineg += 1
    elif word['tag'] == 'I-NSCO':
      insco += 1
    elif word['tag'] == 'I-UNC':
      iunc += 1
    elif word['tag'] == 'I-USCO':
      iusco += 1
    elif word['tag'] == 'E-NEG':
      eneg += 1
    elif word['tag'] == 'E-NSCO':
      ensco += 1
    elif word['tag'] == 'E-UNC':
      eunc += 1
    elif word['tag'] == 'E-USCO':
      eusco += 1

print(f"Total words: {total_words}")

print("\nB-Tags:")
print(f"  B-NEG:  {bneg}")
print(f"  B-NSCO: {bnsco}")
print(f"  B-UNC:  {bunc}")
print(f"  B-USCO: {busco}")

print("\nI-Tags:")
print(f"  I-NEG:  {ineg}")
print(f"  I-NSCO: {insco}")
print(f"  I-UNC:  {iunc}")
print(f"  I-USCO: {iusco}")

print("\nE-Tags:")
print(f"  E-NEG:  {eneg}")
print(f"  E-NSCO: {ensco}")
print(f"  E-UNC:  {eunc}")
print(f"  E-USCO: {eusco}")

print(f"\nO-Tags: {o}")


Total words: 41993

B-Tags:
  B-NEG:  1019
  B-NSCO: 971
  B-UNC:  116
  B-USCO: 115

I-Tags:
  I-NEG:  0
  I-NSCO: 1304
  I-UNC:  3
  I-USCO: 251

E-Tags:
  E-NEG:  18
  E-NSCO: 643
  E-UNC:  54
  E-USCO: 88

O-Tags: 37411


Sentences that have at least one word with a tag different of O and sentences that have just tags with O - Test set

In [124]:
parsed_sentences = []
O_sentences = []
for sentence in sentences:
  diff = False
  for word in sentence:
    if word['tag']!='O':
      diff = True
      break
  if diff:
    parsed_sentences.append(sentence)
  else:
    O_sentences.append(sentence)

In [125]:
print(len(parsed_sentences))
print(len(O_sentences))

3577
7573


Sentences that have at least one word with a tag different of O and sentences that have just tags with O - Train set

In [126]:
parsed_sentences_test = []
O_sentences_test = []
for sentence in sentences_test:
  diff = False
  for word in sentence:
    if word['tag']!='O':
      diff = True
      break
  if diff:
    parsed_sentences_test.append(sentence)
  else:
    O_sentences_test.append(sentence)

In [127]:
print(len(parsed_sentences_test))
print(len(O_sentences_test))
print(len(sentences_test))

971
2088
3059


##Start CRF approach

Get CRF features

This function converts a sentence into a list of labels.

In [128]:
def sent2labels(sent):
    return [word['tag'] for word in sent]

Generate feature dictionaries for words in a sentence for use in CRF models

In [129]:

def get_word_to_crf_features(sentence, i):
    word = sentence[i]['word']
    lemma = sentence[i]['lemma']
    pos = sentence[i]['pos']
    suffix = sentence[i]['suffix']

    features = {
        'bias': 1.0,
        'word.lower()':word.lower(),
        'pos': pos,
        'lemma': lemma,
        'suffix':suffix,
        'word.isdigit()':word.isdigit(),

    }

    # Add features for the previous three words within sentence boundaries
    for j in range(1, 4):
        if i - j >= 0:
            word_prev = sentence[i - j]['word']
            pos_prev = sentence[i - j]['pos']
            lemma_prev = sentence[i - j]['lemma']
            suffix_prev = sentence[i-j]['suffix']
            features.update({
                f'-{j}:word.lower()': word_prev.lower(),
                f'-{j}:pos': pos_prev,
                f'-{j}:lemma': lemma_prev,
                f'-{j}:suffix':suffix_prev,
                f'-{j}:word.isdigit()':word_prev.isdigit(),
            })

    # Add features for the next three words within sentence boundaries
    for j in range(1, 4):
        if i + j < len(sentence):
            word_next = sentence[i + j]['word']
            pos_next = sentence[i + j]['pos']
            lemma_next = sentence[i + j]['lemma']
            suffix_next = sentence[i+j]['suffix']
            features.update({
                f'+{j}:word.lower()': word_next.lower(),
                f'+{j}:pos': pos_next,
                f'+{j}:lemma': lemma_next,
                f'+{j}:suffix':suffix_next,
                f'+{j}:word.isdigit()':word_next.isdigit(),
            })

    if i == 0:
      features['bos'] = True
    else:
      features['bos'] = False

    if i+1 == len(sentence):
      features['eos'] = True
    else:
      features['eos'] = False


    return features


def get_sent_to_crf_features(sentence):
    return [get_word_to_crf_features(sentence, i) for i in range(len(sentence))]


Choose train_sents

In [130]:
train_sents = sentences

Choose test_sents

In [131]:
test_sents = sentences_test

Compute X_train and y_train

Preparing training data for a CRF model by extracting features and labels from sentences

In [132]:

X_train = [get_sent_to_crf_features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]


trainer_crf = crfs.Trainer(verbose=False) # Instance a CRF trainer

for xseq, yseq in zip(X_train, y_train):
    trainer_crf.append(xseq, yseq) # Stack the data


In [133]:
print(X_train[0][0])

{'bias': 1.0, 'word.lower()': 'nº', 'pos': 'NOUN', 'lemma': 'nº', 'suffix': 'nº', 'word.isdigit()': False, '+1:word.lower()': 'historia', '+1:pos': 'NOUN', '+1:lemma': 'historia', '+1:suffix': 'ria', '+1:word.isdigit()': False, '+2:word.lower()': 'clinica', '+2:pos': 'ADJ', '+2:lemma': 'clinico', '+2:suffix': 'ica', '+2:word.isdigit()': False, '+3:word.lower()': 'nºepisodi', '+3:pos': 'PROPN', '+3:lemma': 'nºepisodi', '+3:suffix': 'odi', '+3:word.isdigit()': False, 'bos': True, 'eos': False}


Compute X_test and y_test

In [134]:
X_test = [get_sent_to_crf_features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

Set Parameters for CRF

In [135]:
trainer_crf.set_params({
    'c1': 1.0,   # Coefficient for L1 regularization
    'c2': 1e-3,  # Coefficient for L2 regularization
    'max_iterations': 100,
    'feature.possible_transitions': True
})

Train CRF

In [136]:
trainer_crf.train('npl_ner_crf.crfsuite') # Train the model and save it locally.
tagger_crf = crfs.Tagger()
tagger_crf.open('npl_ner_crf.crfsuite') # Load the inference API

In [137]:
from sklearn.preprocessing import LabelBinarizer
from itertools import chain
from sklearn.metrics import classification_report, confusion_matrix

Generate a classification report for BIO-tagged sequence

In [138]:
from sklearn.preprocessing import LabelBinarizer
def bio_classification_report(y_true, y_pred):
    """

    Classification report.
    You can use this as evaluation for both in the baseline model and new model.

    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))

    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}

    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

Here we generate predictions for the test set using a CRF model.

In [139]:
y_pred_crf = [tagger_crf.tag(x) for x in X_test]
report_crf = bio_classification_report(y_test, y_pred_crf)

print(y_pred_crf)
print("--------")
print(y_test)



[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'B-NSCO', 'E-NSCO', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'B-NSCO', 'E-NSCO', 'O', 'O', 'O', 'O', 'O'], ['O'], ['O', 'O'], ['O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O'], ['O', 'O'], ['B-NSCO', 'B-NEG'], ['O', 'O', 'B-NEG', 'B-NSCO', 'E-NSCO', 'B-NEG', 'B-NSCO', 'B-NSCO', 'I-NSCO', 'I-NSCO', 'I-NSCO', 'E-NSCO', 'B-NEG'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [140]:
print(report_crf)

              precision    recall  f1-score   support

       B-NEG       0.97      0.96      0.97      1019
       E-NEG       0.94      0.89      0.91        18
      B-NSCO       0.95      0.92      0.94       971
      E-NSCO       0.86      0.86      0.86       643
      I-NSCO       0.88      0.88      0.88      1304
       B-UNC       0.91      0.71      0.80       116
       E-UNC       0.87      0.72      0.79        54
       I-UNC       0.00      0.00      0.00         3
      B-USCO       0.91      0.72      0.81       115
      E-USCO       0.61      0.49      0.54        88
      I-USCO       0.69      0.64      0.66       251

   micro avg       0.90      0.87      0.89      4582
   macro avg       0.78      0.71      0.74      4582
weighted avg       0.90      0.87      0.89      4582
 samples avg       0.10      0.10      0.10      4582



Calculate metrics for 'NEG', 'NSCO', 'UNC' and 'USCO'  
Preprocesse the true and predicted BIO-tagged sequences by removing the 'B-' and 'I-' prefixes

In [141]:
prep_y_test = []
prep_y_pred_crf = []
for true_sent , pred_sent in zip(y_test,y_pred_crf):
  prep_true_sent = []
  prep_pred_sent = []
  for true_tag, pred_tag in zip(true_sent,pred_sent):

    if true_tag!='O':
      prep_tag = true_tag[2:]
      prep_true_sent.append(prep_tag)
    else:
      prep_true_sent.append(true_tag)

    if pred_tag!='O':
      prep_tag = pred_tag[2:]
      prep_pred_sent.append(prep_tag)
    else:
      prep_pred_sent.append(pred_tag)
  prep_y_test.append(prep_true_sent)
  prep_y_pred_crf.append(prep_pred_sent)



In [142]:
print(prep_y_test[9])
print(y_test[9])


['O', 'O', 'NEG', 'NSCO', 'O', 'NEG', 'NSCO', 'NSCO', 'NSCO', 'NSCO', 'NSCO', 'NSCO', 'NEG']
['O', 'O', 'B-NEG', 'B-NSCO', 'O', 'B-NEG', 'B-NSCO', 'B-NSCO', 'I-NSCO', 'I-NSCO', 'I-NSCO', 'E-NSCO', 'B-NEG']


In [143]:
report_crf_2 = bio_classification_report(prep_y_test, prep_y_pred_crf)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [144]:
print(report_crf_2)

              precision    recall  f1-score   support

         NEG       0.97      0.96      0.96      1037
        NSCO       0.92      0.91      0.92      2918
         UNC       0.90      0.70      0.79       173
        USCO       0.79      0.69      0.73       454

   micro avg       0.92      0.89      0.90      4582
   macro avg       0.89      0.81      0.85      4582
weighted avg       0.92      0.89      0.90      4582
 samples avg       0.10      0.10      0.10      4582



Compare results

In [145]:
#Compare result for bad sentences tagging
idx = 0
count=0
for true_sent , pred_sent in zip(y_test,y_pred_crf):
  for true_tag, pred_tag in zip(true_sent,pred_sent):
    if true_tag != pred_tag:
      print(idx)
      print("True: ",true_sent)
      print("Pred: ",pred_sent)
      print("---------------")
      count+=1
      break

  idx+=1
print(count)

9
True:  ['O', 'O', 'B-NEG', 'B-NSCO', 'O', 'B-NEG', 'B-NSCO', 'B-NSCO', 'I-NSCO', 'I-NSCO', 'I-NSCO', 'E-NSCO', 'B-NEG']
Pred:  ['O', 'O', 'B-NEG', 'B-NSCO', 'E-NSCO', 'B-NEG', 'B-NSCO', 'B-NSCO', 'I-NSCO', 'I-NSCO', 'I-NSCO', 'E-NSCO', 'B-NEG']
---------------
10
True:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NSCO', 'E-NSCO', 'B-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
---------------
51
True:  ['O', 'B-NSCO', 'B-NEG', 'O', 'O', 'B-NSCO', 'B-NEG', 'B-NSCO', 'B-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
---------------
68
True:  ['O', 'O', 'O', 'B-NEG', 'B-NSCO', 'I-NSCO', 'I-NSCO', 'I-NSCO', 'E-NSCO']
Pred:  ['O', 'O', 'O', 'B-NEG', 'B-NSCO', 'O', 'O', 'O', 'O']
---------------
113
True:  ['O', 'O', '

Tagging example True vs Pred

In [146]:

for y_pred_crf_s, test_sents_s in zip(y_pred_crf[412:416],test_sents[412:416]):
  sent_true = []
  sent_pred = []
  for pred_tag, word in zip(y_pred_crf_s, test_sents_s):
    sent_true.append((word['word'],word['tag']))
    sent_pred.append((word['word'],pred_tag))
  print("True: ",sent_true)
  print("Pred: ",sent_pred)
  print("----------------")

True:  [('tc', 'O'), ('litiasis', 'O'), ('de', 'O'), ('6-7', 'O'), ('mm', 'O'), ('a', 'O'), ('nivel', 'O'), ('del', 'O'), ('ureter', 'O'), ('distal', 'O')]
Pred:  [('tc', 'O'), ('litiasis', 'O'), ('de', 'O'), ('6-7', 'O'), ('mm', 'O'), ('a', 'O'), ('nivel', 'O'), ('del', 'O'), ('ureter', 'O'), ('distal', 'O')]
----------------
True:  [('tumoracion', 'O'), ('probablemente', 'B-UNC'), ('solida', 'B-USCO'), ('en', 'I-USCO'), ('polo', 'I-USCO'), ('superior', 'I-USCO'), ('del', 'I-USCO'), ('riñon', 'I-USCO'), ('izquierdo', 'I-USCO'), ('de', 'I-USCO'), ('2,5', 'I-USCO'), ('cm', 'I-USCO'), ('a', 'I-USCO'), ('completar', 'I-USCO'), ('estudio', 'E-USCO')]
Pred:  [('tumoracion', 'O'), ('probablemente', 'B-UNC'), ('solida', 'B-USCO'), ('en', 'I-USCO'), ('polo', 'I-USCO'), ('superior', 'I-USCO'), ('del', 'I-USCO'), ('riñon', 'I-USCO'), ('izquierdo', 'I-USCO'), ('de', 'I-USCO'), ('2,5', 'I-USCO'), ('cm', 'I-USCO'), ('a', 'I-USCO'), ('completar', 'I-USCO'), ('estudio', 'E-USCO')]
----------------
Tr

Calculate metrics for words

Calculate Precision, Recall and F1  
We calculate the false positives (FP), false negatives (FN), and true positives (TP) for different categories (NEG, UNC, USCO, NSCO)

In [147]:
neg_fp = 0
neg_fn = 0
neg_tp = 0

nsco_fp = 0
nsco_fn = 0
nsco_tp = 0

unc_fp = 0
unc_fn = 0
unc_tp = 0

usco_fp = 0
usco_fn = 0
usco_tp = 0

d = {'NEG' : {'FP' : 0, "FN" : 0, 'TP' : 0}, 'UNC' : {'FP' : 0, "FN" : 0, 'TP' : 0}, 'USCO' : {'FP' : 0, "FN" : 0, 'TP' : 0}, 'NSCO' : {'FP' : 0, "FN" : 0, 'TP' : 0}}

iddd = 0
bla = 0
fn = 0
tp = 0
for p_sen, gt_sen in zip(y_pred_crf, y_test):
  iddd += 1
  for i in range(len(p_sen)):
    category = ""
    # If the prediciton is 'O'
    if p_sen[i] == 'O':
      if gt_sen[i] == 'O' or gt_sen[i][0] == 'I' or gt_sen[i][0] == 'E':
        continue

      category = gt_sen[i][2:]

      d[category]['FN'] += 1
      if category == "NSCO":
        fn += 1
      i += 1

      # Go to the end of the tag. This makes sure that for a sequence of
      # B-NSCO, I-NSCO, E-NSCO only 1 fn is added
      while i < len(p_sen) and gt_sen[i][2:] == category:
        i += 1
      i -= 1

    elif gt_sen[i] == 'O':
      category = p_sen[i][2:]
      d[category]['FP'] += 1
      i += 1
      while i < len(p_sen) and p_sen[i][2:] == category:
        i += 1
      i -= 1

    elif p_sen[i][0] == "B":
      p_category = p_sen[i][2:]
      if gt_sen[i] != p_sen[i]:
        bla += 1
        gt_category = gt_sen[i][2:]
        d[p_category]['FP'] += 1
        d[gt_category]['FN'] += 1
        i += 1
        while i < len(p_sen) and (gt_sen[i][2:] == gt_category or p_sen[i][2:] == p_category):
          i += 1
        i -= 1


      elif gt_sen[i] == p_sen[i]:
        i += 1
        good = True
        while i < len(p_sen):
          if gt_sen[i][2:] == 'O' and p_sen[i][2:] == 'O':
            i -= 1
            break
          if gt_sen[i][2:] != p_category and p_sen[i][2:] != p_category:
            i -= 1
            break
          if gt_sen[i][2:] == p_category and p_sen[i][2:] == p_category:
            i += 1
            continue

          good = False
          d[p_category]['FN'] += 1
          if gt_sen[i][2:] != p_category:
            pass
          elif p_sen[i][2:] != p_category:
            pass
          else:
            print(f'{gt_sen[i]=} {p_sen[i]=}')
         # i-=1
          break
        if good:
          if p_category == "NSCO":
            tp += 1
          d[p_category]["TP"] += 1


In [148]:
for tag, metrics in d.items():
  for metric, nr in metrics.items():
    print(f'{tag}: {metric} {nr}')

NEG: FP 27
NEG: FN 43
NEG: TP 974
UNC: FP 10
UNC: FN 37
UNC: TP 82
USCO: FP 63
USCO: FN 61
USCO: TP 55
NSCO: FP 218
NSCO: FN 146
NSCO: TP 823


In [149]:
precision = []
recall = []
f1 = []

for tag, metrics in d.items():
  p = metrics['TP'] / (metrics['TP'] + metrics['FP'])
  precision.append((tag, p))
  r = metrics['TP'] / (metrics['TP'] + metrics['FN'])
  recall.append((tag, r))
  f1.append((tag, (2 * p * r) / (p + r)))

print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F1: " + str(f1))

Precision: [('NEG', 0.973026973026973), ('UNC', 0.8913043478260869), ('USCO', 0.4661016949152542), ('NSCO', 0.7905859750240154)]
Recall: [('NEG', 0.9577187807276303), ('UNC', 0.6890756302521008), ('USCO', 0.47413793103448276), ('NSCO', 0.849329205366357)]
F1: [('NEG', 0.9653121902874132), ('UNC', 0.7772511848341233), ('USCO', 0.47008547008547), ('NSCO', 0.8189054726368159)]
